In [1]:
import keras.applications.resnet50 as resnet
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

#디렉토리 생성용
import os

Using TensorFlow backend.


In [2]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16
DROPOUT = 0.5

# 이미지 volume 값
input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH)

# 모델 받아오기
model_origin = resnet.ResNet50(include_top=False, weights=None, input_shape=input_shape, pooling='avg', classes=LABEL_NUM)

# 모델 dropout 설정
output = model_origin.output

output = Dense(2048, activation="relu")(output)
output = Dropout(DROPOUT)(output)
output = Dense(LABEL_NUM, activation='softmax', name='predictions')(output)

model = Model(model_origin.input, output)

# 모델 출력
model.summary()

# Compling
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:
# Validation값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping(patience=2)

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
'''
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
'''

#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/ResNet/" + NOW)

#디렉토리 생성
def MakeDirectory(dir_path):
    try:
        if not(os.path.isdir(dir_path)):
            os.makedirs(os.path.join(dir_path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

In [ ]:
#훈련 6개의 데이터 셋으로 진행(Validation 데이터 셋이 다름)
DATASET_NUM = 6
EPOCH_NUM = 20
BATCH_SIZE = 32
CYCLE_NUM = 8

#훈련용 이미지 생성기
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

# Weight, Model 저장용 폴더 생성
model_save_dir = "./Models/ResNet/" + NOW
MakeDirectory(model_save_dir)

#모델 저장
model_json = model.to_json()
with open(model_save_dir + "/model_resnet_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)

# Weight 로드(안해도됨)
#model.load_weights("./Models/ResNet/20181214_11_14_03/weight_resnet_20181214_18_12_43.h5")
             
#몇 사이클 돌지
for cycle in range(0, CYCLE_NUM):
    
    print("Cycle ", (cycle + 1))
    
    # 데이터셋 수 만큼 훈련
    for index in range(1, (DATASET_NUM + 1)):

        print("Training ", index)

        # 데이터 셋 생성
        dir_path = "".join(['./Dataset_Multi/Dataset', str(index)])
        training_set = train_datagen.flow_from_directory(dir_path + '/Train',
                                                    target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                    batch_size = BATCH_SIZE)

        validation_set = validation_datagen.flow_from_directory(dir_path + '/Validation',
                                                   target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                   batch_size = BATCH_SIZE)

        #배치 수 계산
        train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
        validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)
        
        #훈련!
        model.fit_generator(training_set,
                            steps_per_epoch = train_step_epoch,
                            epochs = EPOCH_NUM,
                            validation_data = validation_set,
                            validation_steps = validation_step_epoch,
                            #class_weight=class_weights,
                            shuffle=True,
                            callbacks=[tensorboard, early_stopping])

        print("Traing ", index, " done")
        
        #오늘 날짜_시간
        current_time = datetime.now().strftime('%Y%m%d_%H_%M_%S')
        
        # Weight 저장
        model.save_weights(model_save_dir + "/weight_resnet_" + current_time + ".h5")
        print("Saved model to disk\n")

Cycle  1
Training  1
Found 62573 images belonging to 16 classes.
Found 15875 images belonging to 16 classes.
Epoch 1/20
1955/1955 [==============================] - 867s 443ms/step - loss: 2.9740 - acc: 0.2579 - val_loss: 12.0126 - val_acc: 0.2528
Epoch 2/20
1955/1955 [==============================] - 310s 159ms/step - loss: 2.5499 - acc: 0.2609 - val_loss: 2.4979 - val_acc: 0.2532
Epoch 3/20
1955/1955 [==============================] - 310s 158ms/step - loss: 2.4393 - acc: 0.2619 - val_loss: 2.4495 - val_acc: 0.2530
Epoch 4/20
1955/1955 [==============================] - 310s 158ms/step - loss: 2.4243 - acc: 0.2624 - val_loss: 2.4798 - val_acc: 0.2538
Epoch 5/20
1955/1955 [==============================] - 310s 158ms/step - loss: 2.4054 - acc: 0.2643 - val_loss: 2.5212 - val_acc: 0.2529
Traing  1  done
Saved model to disk

Training  2
Found 68806 images belonging to 16 classes.
Found 9642 images belonging to 16 classes.
Epoch 1/20
2150/2150 [==============================] - 826s 384

1955/1955 [==============================] - 311s 159ms/step - loss: 0.3594 - acc: 0.8751 - val_loss: 0.4754 - val_acc: 0.9035
Epoch 2/20
1955/1955 [==============================] - 311s 159ms/step - loss: 0.3368 - acc: 0.8834 - val_loss: 0.4841 - val_acc: 0.8931
Epoch 3/20
1955/1955 [==============================] - 311s 159ms/step - loss: 0.3220 - acc: 0.8861 - val_loss: 0.3104 - val_acc: 0.9229
Epoch 4/20
1955/1955 [==============================] - 311s 159ms/step - loss: 0.3079 - acc: 0.8932 - val_loss: 0.4940 - val_acc: 0.8780
Epoch 5/20
1955/1955 [==============================] - 311s 159ms/step - loss: 0.3086 - acc: 0.8933 - val_loss: 0.5605 - val_acc: 0.8894
Traing  1  done
Saved model to disk

Training  2
Found 68806 images belonging to 16 classes.
Found 9642 images belonging to 16 classes.
Epoch 1/20
2150/2150 [==============================] - 332s 155ms/step - loss: 0.3236 - acc: 0.8868 - val_loss: 0.3046 - val_acc: 0.9444
Epoch 2/20
2150/2150 [=========================

In [16]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset_Multi/Testset',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

Found 9588 images belonging to 16 classes.
-- Evaluate --
acc: 19.03%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 1.000]


In [7]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])

-- Evaluate --
acc: 34.58%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.042 0.005 0.000 0.000 0.001 0.002 0.001 0.002 0.046 0.003 0.000 0.003
 0.049 0.006 0.047 0.794]
